## Imports and dataset loading

In [1]:
import sys
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, cross_validate
from sklearn.neural_network import MLPClassifier
#from sklearn.metrics import F1Score
import tensorflow as tf
import matplotlib as plt
from tensorflow import keras
import datetime

In [2]:
DATASET_PATH: str = "../../dataset/"

In [3]:
df = pd.read_csv('../../dataset/trainU.csv', sep=',', index_col=0)

## Train-val-test split

In [4]:
data_cols = [col_name for col_name in df.columns if col_name != 'DISEASE']

In [5]:
train_x, val_x, train_y, val_y = train_test_split(df[data_cols], df['DISEASE'], test_size=.25, stratify=df['DISEASE'])

In [6]:
print("Train: ", train_x.shape, train_y.shape, "\nVal: ", val_x.shape, val_y.shape)

Train:  (224, 4420) (224,) 
Val:  (75, 4420) (75,)


In [7]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()

In [8]:
encoder = encoder.fit(np.array(df['DISEASE']).reshape(-1, 1))

In [9]:
train_y_o = encoder.transform(np.asarray(train_y).reshape(-1,1))
val_y_o   = encoder.transform(np.asarray(val_y).reshape(-1,1))

In [10]:
print("Train: ", train_x.shape, train_y_o.shape, "\nVal: ", val_x.shape, val_y_o.shape)

Train:  (224, 4420) (224, 5) 
Val:  (75, 4420) (75, 5)


## Board

In [11]:
rm -rf ./logs/

In [12]:
tf.random.set_seed(42)

In [13]:
%load_ext tensorboard

In [14]:
from tensorboard.plugins.hparams import api as hp

HP_NUM_UNITS1 = hp.HParam('num_units1', hp.Discrete([64, 56]))
HP_NUM_UNITS2 = hp.HParam('num_units2', hp.Discrete([32, 24]))
HP_NUM_UNITS3 = hp.HParam('num_units3', hp.Discrete([16, 8]))
HP_DROPOUT1 = hp.HParam('dropout1', hp.RealInterval(0.2, 0.4))
HP_DROPOUT2 = hp.HParam('dropout2', hp.RealInterval(0.1, 0.3))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

EPOCHS = 200
METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
    hp.hparams_config(
        hparams=[HP_NUM_UNITS1, HP_NUM_UNITS2, HP_NUM_UNITS3, HP_DROPOUT1, HP_DROPOUT2, HP_OPTIMIZER],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
        )

2022-06-20 19:25:59.010801: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [15]:
def train_val_model(hparams, run_name):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(hparams[HP_NUM_UNITS1], activation=tf.nn.relu),
        tf.keras.layers.Dropout(hparams[HP_DROPOUT1]),
        tf.keras.layers.Dense(hparams[HP_NUM_UNITS2], activation=tf.nn.relu),
        tf.keras.layers.Dropout(hparams[HP_DROPOUT1]),
        tf.keras.layers.Dense(hparams[HP_NUM_UNITS2], activation=tf.nn.relu),
        tf.keras.layers.Dropout(hparams[HP_DROPOUT2]),
        tf.keras.layers.Dense(5, activation=tf.nn.softmax),
        ])
    model.compile(
      optimizer=hparams[HP_OPTIMIZER],
      loss='categorical_crossentropy',
      metrics=['accuracy'],
    )

    log_dir = "logs/fit/" + run_name
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    
    model.fit(train_x, train_y_o.toarray(),   
            epochs=EPOCHS,
            # concettualmente e' sbagliato utilizzare questo VAL_SET qui, ma ci serve solo per
            # controllare l'eventuale regime di overfitting nel plot loss&accuracy nel secondo tensorboard 
            validation_data=(val_x, val_y_o.toarray()),
            callbacks=[tensorboard_callback]
            )
    _, accuracy = model.evaluate(val_x, val_y_o.toarray())
    
    res = [accuracy, num_units1, num_units2, num_units3, dropout1_rate, dropout2_rate, optimizer]
    
    return res

In [16]:
def run(run_dir, hparams, run_name):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        results_run = train_val_model(hparams, run_name)
        tf.summary.scalar(METRIC_ACCURACY, results_run[0], step=1)
    return results_run

In [17]:
session_num = 0
results_session = []

for num_units1 in HP_NUM_UNITS1.domain.values:
    for num_units2 in HP_NUM_UNITS2.domain.values:
        for num_units3 in HP_NUM_UNITS3.domain.values:
            for dropout1_rate in (HP_DROPOUT1.domain.min_value, HP_DROPOUT1.domain.max_value):
                for dropout2_rate in (HP_DROPOUT2.domain.min_value, HP_DROPOUT2.domain.max_value):
                    for optimizer in HP_OPTIMIZER.domain.values:
                        hparams = {
                          HP_NUM_UNITS1: num_units1,
                          HP_NUM_UNITS2: num_units2,
                          HP_NUM_UNITS3: num_units3,
                          HP_DROPOUT1: dropout1_rate,
                          HP_DROPOUT2: dropout2_rate,
                          HP_OPTIMIZER: optimizer,
                        }
                        run_name = "run-%d" % session_num
                        print('--- Starting trial: %s' % run_name)
                        print({h.name: hparams[h] for h in hparams})
                        results_session.append(run('logs/hparam_tuning/' + run_name, hparams, run_name))
                        #if session_num != 0:
                        #    if results_session[session_num][0] > results_session[session_num-1]:
                                    #save model
                        #else:
                        #    continue
                        session_num += 1

--- Starting trial: run-0
{'num_units1': 56, 'num_units2': 24, 'num_units3': 8, 'dropout1': 0.2, 'dropout2': 0.1, 'optimizer': 'adam'}
Epoch 1/200
7/7 [==============================] - 1s 64ms/step - loss: 1.7221 - accuracy: 0.3036 - val_loss: 1.4312 - val_accuracy: 0.4000
Epoch 2/200
7/7 [==============================] - 0s 23ms/step - loss: 1.4797 - accuracy: 0.3661 - val_loss: 1.3915 - val_accuracy: 0.4400
Epoch 3/200
7/7 [==============================] - 0s 23ms/step - loss: 1.5056 - accuracy: 0.3348 - val_loss: 1.3413 - val_accuracy: 0.4400
Epoch 4/200
7/7 [==============================] - 0s 23ms/step - loss: 1.3874 - accuracy: 0.3973 - val_loss: 1.1823 - val_accuracy: 0.7600
Epoch 5/200
7/7 [==============================] - 0s 23ms/step - loss: 1.2608 - accuracy: 0.5045 - val_loss: 1.1469 - val_accuracy: 0.6133
Epoch 6/200
7/7 [==============================] - 0s 24ms/step - loss: 1.2011 - accuracy: 0.5670 - val_loss: 1.0389 - val_accuracy: 0.7600
Epoch 7/200
7/7 [========

7/7 [==============================] - 0s 24ms/step - loss: 0.1844 - accuracy: 0.9018 - val_loss: 0.3989 - val_accuracy: 0.8933
Epoch 59/200
7/7 [==============================] - 0s 22ms/step - loss: 0.1880 - accuracy: 0.9241 - val_loss: 0.2476 - val_accuracy: 0.9333
Epoch 60/200
7/7 [==============================] - 0s 22ms/step - loss: 0.2039 - accuracy: 0.9196 - val_loss: 0.4705 - val_accuracy: 0.9067
Epoch 61/200
7/7 [==============================] - 0s 23ms/step - loss: 0.1567 - accuracy: 0.9375 - val_loss: 0.2699 - val_accuracy: 0.9200
Epoch 62/200
7/7 [==============================] - 0s 24ms/step - loss: 0.1578 - accuracy: 0.9330 - val_loss: 0.1817 - val_accuracy: 0.9467
Epoch 63/200
7/7 [==============================] - 0s 23ms/step - loss: 0.1515 - accuracy: 0.9375 - val_loss: 0.5011 - val_accuracy: 0.8933
Epoch 64/200
7/7 [==============================] - 0s 24ms/step - loss: 0.2005 - accuracy: 0.9062 - val_loss: 0.3660 - val_accuracy: 0.9200
Epoch 65/200
7/7 [========

7/7 [==============================] - 0s 22ms/step - loss: 0.1009 - accuracy: 0.9643 - val_loss: 0.5788 - val_accuracy: 0.9067
Epoch 117/200
7/7 [==============================] - 0s 23ms/step - loss: 0.1115 - accuracy: 0.9554 - val_loss: 0.4819 - val_accuracy: 0.9067
Epoch 118/200
7/7 [==============================] - 0s 22ms/step - loss: 0.1321 - accuracy: 0.9375 - val_loss: 0.5397 - val_accuracy: 0.8933
Epoch 119/200
7/7 [==============================] - 0s 22ms/step - loss: 0.1078 - accuracy: 0.9509 - val_loss: 0.4757 - val_accuracy: 0.9067
Epoch 120/200
7/7 [==============================] - 0s 23ms/step - loss: 0.0979 - accuracy: 0.9598 - val_loss: 0.7308 - val_accuracy: 0.8933
Epoch 121/200
7/7 [==============================] - 0s 23ms/step - loss: 0.1533 - accuracy: 0.9241 - val_loss: 0.2633 - val_accuracy: 0.9200
Epoch 122/200
7/7 [==============================] - 0s 24ms/step - loss: 0.1284 - accuracy: 0.9509 - val_loss: 0.3217 - val_accuracy: 0.8933
Epoch 123/200
7/7 [=

Epoch 174/200
7/7 [==============================] - 0s 23ms/step - loss: 0.0581 - accuracy: 0.9821 - val_loss: 0.5976 - val_accuracy: 0.9067
Epoch 175/200
7/7 [==============================] - 0s 23ms/step - loss: 0.1146 - accuracy: 0.9643 - val_loss: 0.4907 - val_accuracy: 0.9067
Epoch 176/200
7/7 [==============================] - 0s 22ms/step - loss: 0.0935 - accuracy: 0.9732 - val_loss: 0.6402 - val_accuracy: 0.8933
Epoch 177/200
7/7 [==============================] - 0s 23ms/step - loss: 0.0539 - accuracy: 0.9821 - val_loss: 0.5387 - val_accuracy: 0.9067
Epoch 178/200
7/7 [==============================] - 0s 22ms/step - loss: 0.1561 - accuracy: 0.9464 - val_loss: 0.3115 - val_accuracy: 0.8933
Epoch 179/200
7/7 [==============================] - 0s 22ms/step - loss: 0.1518 - accuracy: 0.9375 - val_loss: 1.7835 - val_accuracy: 0.8000
Epoch 180/200
7/7 [==============================] - 0s 21ms/step - loss: 0.2979 - accuracy: 0.8884 - val_loss: 0.3506 - val_accuracy: 0.8800
Epoch 

7/7 [==============================] - 0s 22ms/step - loss: 0.6612 - accuracy: 0.7411 - val_loss: 0.6142 - val_accuracy: 0.7600
Epoch 31/200
7/7 [==============================] - 0s 22ms/step - loss: 0.5992 - accuracy: 0.7679 - val_loss: 0.6010 - val_accuracy: 0.7600
Epoch 32/200
7/7 [==============================] - 0s 23ms/step - loss: 0.5869 - accuracy: 0.7768 - val_loss: 0.5979 - val_accuracy: 0.7733
Epoch 33/200
7/7 [==============================] - 0s 22ms/step - loss: 0.5880 - accuracy: 0.7902 - val_loss: 0.5894 - val_accuracy: 0.7733
Epoch 34/200
7/7 [==============================] - 0s 22ms/step - loss: 0.5596 - accuracy: 0.7857 - val_loss: 0.6054 - val_accuracy: 0.7600
Epoch 35/200
7/7 [==============================] - 0s 23ms/step - loss: 0.6202 - accuracy: 0.7723 - val_loss: 0.5859 - val_accuracy: 0.7600
Epoch 36/200
7/7 [==============================] - 0s 21ms/step - loss: 0.5547 - accuracy: 0.8170 - val_loss: 0.5706 - val_accuracy: 0.7733
Epoch 37/200
7/7 [========

...

7/7 [==============================] - 0s 23ms/step - loss: 0.8294 - accuracy: 0.7143 - val_loss: 0.8838 - val_accuracy: 0.7467
Epoch 122/200
7/7 [==============================] - 0s 23ms/step - loss: 0.9166 - accuracy: 0.6696 - val_loss: 0.8674 - val_accuracy: 0.7467
Epoch 123/200
7/7 [==============================] - 0s 23ms/step - loss: 0.8783 - accuracy: 0.6920 - val_loss: 0.9229 - val_accuracy: 0.7467
Epoch 124/200
7/7 [==============================] - 0s 24ms/step - loss: 0.8732 - accuracy: 0.6875 - val_loss: 0.8767 - val_accuracy: 0.7467
Epoch 125/200
7/7 [==============================] - 0s 23ms/step - loss: 0.8165 - accuracy: 0.7143 - val_loss: 0.8726 - val_accuracy: 0.7467
Epoch 126/200
7/7 [==============================] - 0s 23ms/step - loss: 0.8453 - accuracy: 0.7098 - val_loss: 0.9010 - val_accuracy: 0.7467
Epoch 127/200
7/7 [==============================] - 0s 23ms/step - loss: 0.8464 - accuracy: 0.6964 - val_loss: 0.9353 - val_accuracy: 0.7467
Epoch 128/200
7/7 [=

Epoch 179/200
7/7 [==============================] - 0s 23ms/step - loss: 0.8316 - accuracy: 0.7143 - val_loss: 0.8916 - val_accuracy: 0.7467
Epoch 180/200
7/7 [==============================] - 0s 22ms/step - loss: 0.9417 - accuracy: 0.6607 - val_loss: 0.8429 - val_accuracy: 0.7467
Epoch 181/200
7/7 [==============================] - 0s 23ms/step - loss: 0.8746 - accuracy: 0.6920 - val_loss: 0.8515 - val_accuracy: 0.7467
Epoch 182/200
7/7 [==============================] - 0s 23ms/step - loss: 0.8775 - accuracy: 0.6830 - val_loss: 0.9609 - val_accuracy: 0.7467
Epoch 183/200
7/7 [==============================] - 0s 22ms/step - loss: 0.8909 - accuracy: 0.6786 - val_loss: 0.9874 - val_accuracy: 0.7467
Epoch 184/200
7/7 [==============================] - 0s 23ms/step - loss: 0.8791 - accuracy: 0.6875 - val_loss: 0.8096 - val_accuracy: 0.7467
Epoch 185/200
7/7 [==============================] - 0s 24ms/step - loss: 0.8787 - accuracy: 0.6920 - val_loss: 0.7912 - val_accuracy: 0.7333
Epoch 

7/7 [==============================] - 0s 22ms/step - loss: 0.7681 - accuracy: 0.7054 - val_loss: 0.6802 - val_accuracy: 0.7600
Epoch 36/200
7/7 [==============================] - 0s 25ms/step - loss: 0.7376 - accuracy: 0.7321 - val_loss: 0.6775 - val_accuracy: 0.7600
Epoch 37/200
7/7 [==============================] - 0s 23ms/step - loss: 0.7614 - accuracy: 0.6920 - val_loss: 0.6726 - val_accuracy: 0.7600
Epoch 38/200
7/7 [==============================] - 0s 23ms/step - loss: 0.7352 - accuracy: 0.7232 - val_loss: 0.6485 - val_accuracy: 0.7600
Epoch 39/200
7/7 [==============================] - 0s 22ms/step - loss: 0.6672 - accuracy: 0.7277 - val_loss: 0.6207 - val_accuracy: 0.7600
Epoch 40/200
7/7 [==============================] - 0s 22ms/step - loss: 0.7412 - accuracy: 0.7143 - val_loss: 0.6130 - val_accuracy: 0.7600
Epoch 41/200
7/7 [==============================] - 0s 23ms/step - loss: 0.7381 - accuracy: 0.6964 - val_loss: 0.6508 - val_accuracy: 0.7867
Epoch 42/200
7/7 [========

7/7 [==============================] - 0s 23ms/step - loss: 0.3822 - accuracy: 0.8170 - val_loss: 0.4940 - val_accuracy: 0.8267
Epoch 94/200
7/7 [==============================] - 0s 23ms/step - loss: 0.3839 - accuracy: 0.8482 - val_loss: 0.4160 - val_accuracy: 0.8133
Epoch 95/200
7/7 [==============================] - 0s 23ms/step - loss: 0.4249 - accuracy: 0.8348 - val_loss: 0.5086 - val_accuracy: 0.8133
Epoch 96/200
7/7 [==============================] - 0s 22ms/step - loss: 0.3643 - accuracy: 0.8393 - val_loss: 0.4783 - val_accuracy: 0.8533
Epoch 97/200
7/7 [==============================] - 0s 22ms/step - loss: 0.4319 - accuracy: 0.8214 - val_loss: 0.4462 - val_accuracy: 0.8533
Epoch 98/200
7/7 [==============================] - 0s 24ms/step - loss: 0.3629 - accuracy: 0.8661 - val_loss: 0.5091 - val_accuracy: 0.8133
Epoch 99/200
7/7 [==============================] - 0s 23ms/step - loss: 0.3665 - accuracy: 0.8527 - val_loss: 0.6057 - val_accuracy: 0.8000
Epoch 100/200
7/7 [=======

Epoch 151/200
7/7 [==============================] - 0s 23ms/step - loss: 0.2652 - accuracy: 0.8929 - val_loss: 0.5280 - val_accuracy: 0.8800
Epoch 152/200
7/7 [==============================] - 0s 24ms/step - loss: 0.2392 - accuracy: 0.9152 - val_loss: 0.4774 - val_accuracy: 0.8800
Epoch 153/200
7/7 [==============================] - 0s 23ms/step - loss: 0.2407 - accuracy: 0.9330 - val_loss: 0.5408 - val_accuracy: 0.8800
Epoch 154/200
7/7 [==============================] - 0s 23ms/step - loss: 0.2431 - accuracy: 0.9107 - val_loss: 0.5028 - val_accuracy: 0.8800
Epoch 155/200
7/7 [==============================] - 0s 22ms/step - loss: 0.2469 - accuracy: 0.9241 - val_loss: 0.4687 - val_accuracy: 0.8933
Epoch 156/200
7/7 [==============================] - 0s 24ms/step - loss: 0.2639 - accuracy: 0.9018 - val_loss: 0.4962 - val_accuracy: 0.8800
Epoch 157/200
7/7 [==============================] - 0s 23ms/step - loss: 0.2528 - accuracy: 0.9152 - val_loss: 0.3642 - val_accuracy: 0.8933
Epoch 

In [18]:
%tensorboard --logdir logs/hparam_tuning

In [19]:
%tensorboard --logdir logs/fit

### Custom F1 score (still need to generalize for one hot), TO DO

import keras.backend as K

def f1_score(y_true, y_pred):

    # Count positive samples.
    c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    c2 = K.sum(K.round(K.clip(y_pred, 0, 1)))
    c3 = K.sum(K.round(K.clip(y_true, 0, 1)))

    # If there are no true samples, fix the F1 score at 0.
    if c3 == 0:
        return 0

    # How many selected items are relevant?
    precision = c1 / c2

    # How many relevant items are selected?
    recall = c1 / c3

    # Calculate f1_score
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score


## Model Selection

In [20]:
results_session

[[0.9066666960716248, 56, 24, 8, 0.2, 0.1, 'adam'],
 [0.9066666960716248, 56, 24, 8, 0.2, 0.1, 'sgd'],
 [0.9066666960716248, 56, 24, 8, 0.2, 0.3, 'adam'],
 [0.9066666960716248, 56, 24, 8, 0.2, 0.3, 'sgd'],
 [0.8399999737739563, 56, 24, 8, 0.4, 0.1, 'adam'],
 [0.8933333158493042, 56, 24, 8, 0.4, 0.1, 'sgd'],
 [0.800000011920929, 56, 24, 8, 0.4, 0.3, 'adam'],
 [0.8533333539962769, 56, 24, 8, 0.4, 0.3, 'sgd'],
 [0.9066666960716248, 56, 24, 16, 0.2, 0.1, 'adam'],
 [0.8933333158493042, 56, 24, 16, 0.2, 0.1, 'sgd'],
 [0.7866666913032532, 56, 24, 16, 0.2, 0.3, 'adam'],
 [0.9066666960716248, 56, 24, 16, 0.2, 0.3, 'sgd'],
 [0.8533333539962769, 56, 24, 16, 0.4, 0.1, 'adam'],
 [0.8399999737739563, 56, 24, 16, 0.4, 0.1, 'sgd'],
 [0.800000011920929, 56, 24, 16, 0.4, 0.3, 'adam'],
 [0.8266666531562805, 56, 24, 16, 0.4, 0.3, 'sgd'],
 [0.9200000166893005, 56, 32, 8, 0.2, 0.1, 'adam'],
 [0.9200000166893005, 56, 32, 8, 0.2, 0.1, 'sgd'],
 [0.7866666913032532, 56, 32, 8, 0.2, 0.3, 'adam'],
 [0.93333333730

In [21]:
def scalar_to_ohe(prediction):
    for row in range(prediction.shape[0]):
    #    print(max(prediction[row]))
        a,b,c,d,e = prediction[row]
        if a==max(a,b,c,d,e):
            prediction[row][0] = 1
            prediction[row][1:5] = 0
        elif b==max(a,b,c,d,e):
            prediction[row][1] = 1
            prediction[row][0] = 0
            prediction[row][2:5] = 0
        elif c==max(a,b,c,d,e):
            prediction[row][2] = 1
            prediction[row][0:2] = 0
            prediction[row][3:5] = 0
        elif d==max(a,b,c,d,e):
            prediction[row][3] = 1
            prediction[row][0:3] = 0
            prediction[row][4] = 0
        else:
            prediction[row][4] = 1
            prediction[row][0:4] = 0
    return prediction

In [22]:
best_model = np.argmax([item[0] for item in results_session])

In [23]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(results_session[best_model][1], activation=tf.nn.relu),
        tf.keras.layers.Dropout(results_session[best_model][4]),
        tf.keras.layers.Dense(results_session[best_model][2], activation=tf.nn.relu),
        tf.keras.layers.Dropout(results_session[best_model][5]),
        tf.keras.layers.Dense(results_session[best_model][3], activation=tf.nn.relu),
        tf.keras.layers.Dropout(results_session[best_model][5]),
        tf.keras.layers.Dense(5, activation=tf.nn.softmax),
        ])

model.compile(
  optimizer=results_session[best_model][6],
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

model.fit(train_x, train_y_o.toarray(),   
        epochs=EPOCHS,
         )

val_prediction = model.predict(val_x)

Epoch 1/200
7/7 [==============================] - 1s 4ms/step - loss: 1.5833 - accuracy: 0.2321
Epoch 2/200
7/7 [==============================] - 0s 5ms/step - loss: 1.5678 - accuracy: 0.4062
Epoch 3/200
7/7 [==============================] - 0s 4ms/step - loss: 1.3794 - accuracy: 0.4420
Epoch 4/200
7/7 [==============================] - 0s 4ms/step - loss: 1.3383 - accuracy: 0.4777
Epoch 5/200
7/7 [==============================] - 0s 4ms/step - loss: 1.2668 - accuracy: 0.5357
Epoch 6/200
7/7 [==============================] - 0s 4ms/step - loss: 1.2589 - accuracy: 0.5848
Epoch 7/200
7/7 [==============================] - 0s 4ms/step - loss: 1.1536 - accuracy: 0.5848
Epoch 8/200
7/7 [==============================] - 0s 4ms/step - loss: 1.0883 - accuracy: 0.6250
Epoch 9/200
7/7 [==============================] - 0s 5ms/step - loss: 1.0642 - accuracy: 0.6384
Epoch 10/200
7/7 [==============================] - 0s 4ms/step - loss: 0.9915 - accuracy: 0.6473
Epoch 11/200
7/7 [===========

7/7 [==============================] - 0s 5ms/step - loss: 0.0438 - accuracy: 0.9821
Epoch 85/200
7/7 [==============================] - 0s 4ms/step - loss: 0.0461 - accuracy: 0.9821
Epoch 86/200
7/7 [==============================] - 0s 5ms/step - loss: 0.0696 - accuracy: 0.9643
Epoch 87/200
7/7 [==============================] - 0s 4ms/step - loss: 0.0594 - accuracy: 0.9777
Epoch 88/200
7/7 [==============================] - 0s 4ms/step - loss: 0.0677 - accuracy: 0.9688
Epoch 89/200
7/7 [==============================] - 0s 4ms/step - loss: 0.0880 - accuracy: 0.9688
Epoch 90/200
7/7 [==============================] - 0s 4ms/step - loss: 0.0905 - accuracy: 0.9643
Epoch 91/200
7/7 [==============================] - 0s 4ms/step - loss: 0.0566 - accuracy: 0.9866
Epoch 92/200
7/7 [==============================] - 0s 5ms/step - loss: 0.0538 - accuracy: 0.9821
Epoch 93/200
7/7 [==============================] - 0s 4ms/step - loss: 0.1378 - accuracy: 0.9554
Epoch 94/200
7/7 [===============

7/7 [==============================] - 0s 5ms/step - loss: 0.0887 - accuracy: 0.9688
Epoch 168/200
7/7 [==============================] - 0s 5ms/step - loss: 0.0505 - accuracy: 0.9777
Epoch 169/200
7/7 [==============================] - 0s 5ms/step - loss: 0.0544 - accuracy: 0.9777
Epoch 170/200
7/7 [==============================] - 0s 4ms/step - loss: 0.0577 - accuracy: 0.9777
Epoch 171/200
7/7 [==============================] - 0s 4ms/step - loss: 0.0478 - accuracy: 0.9866
Epoch 172/200
7/7 [==============================] - 0s 4ms/step - loss: 0.0669 - accuracy: 0.9777
Epoch 173/200
7/7 [==============================] - 0s 4ms/step - loss: 0.1174 - accuracy: 0.9509
Epoch 174/200
7/7 [==============================] - 0s 4ms/step - loss: 0.0824 - accuracy: 0.9643
Epoch 175/200
7/7 [==============================] - 0s 4ms/step - loss: 0.0709 - accuracy: 0.9688
Epoch 176/200
7/7 [==============================] - 0s 5ms/step - loss: 0.0616 - accuracy: 0.9777
Epoch 177/200
7/7 [=====

In [24]:
val_prediction_ohe = scalar_to_ohe(val_prediction)

In [25]:
from sklearn.metrics import confusion_matrix

confusion_matrix(encoder.inverse_transform(val_y_o.toarray()), encoder.inverse_transform(val_prediction_ohe))

array([[24,  0,  0,  0,  0],
       [ 0, 32,  0,  1,  0],
       [ 0,  3,  3,  0,  0],
       [ 0,  0,  0,  5,  0],
       [ 0,  4,  0,  0,  3]])

In [26]:
from sklearn.metrics import classification_report

print(classification_report(encoder.inverse_transform(val_y_o.toarray()), encoder.inverse_transform(val_prediction_ohe),zero_division=0))

              precision    recall  f1-score   support

         A1A       1.00      1.00      1.00        24
     DIABETE       0.82      0.97      0.89        33
          GS       1.00      0.50      0.67         6
         MCM       0.83      1.00      0.91         5
        NALD       1.00      0.43      0.60         7

    accuracy                           0.89        75
   macro avg       0.93      0.78      0.81        75
weighted avg       0.91      0.89      0.88        75



Feature Selection (random forest, correlation analysis, recursive feature elimination) [https://www.kaggle.com/code/arrohit/feature-selection-and-hypertuning-svm/notebook]